# Data Preparation

In [1]:
import sqlite3 as sql
import pandas as pd

## Step 1
- You will get a SQLite .db file from us. This already contains data.
- Connect to the db in your language/environment.

In [2]:
connection = sql.connect("the.db")

- Explore the data in it (schema).
- Load the data from the db into your environment.

In [3]:
data_frame = pd.read_sql_query("SELECT * FROM dwd;", connection)
data_frame

,timestamp,id,temperature,temperature_ground,temperature_dew,humidity,airpressure
0,201909190000,1048,7.2,5.3,3.9,79.6,997.2
1,201909190010,1048,7.1,5.4,3.8,79.6,997.2
2,201909190020,1048,7.1,5.4,3.8,79.5,997.3
3,201909190030,1048,7.3,5.6,3.8,78.6,997.3
4,201909190040,1048,7.1,5.3,3.7,79.1,997.3
...,...,...,...,...,...,...,...
76,201909191240,1048,13.4,17.9,3.7,51.9,1000.1
77,201909191250,1048,13.3,16.7,3.0,49.8,1000.1
78,201909191300,1048,13.6,17.5,3.1,49.2,1000.0
79,201909191310,1048,14.0,18.0,3.2,48.0,1000.1


## Step 2
- Find the temperature data from the last timestamp in the data to 15.10.2019 given the data in db

In [4]:
last_original_row_timestamp = data_frame['timestamp'].max()
last_original_row_timestamp

'201909191320'

- https://opendata.dwd.de/
- Download the data for evaluation, but also implement a download in your code.

In [5]:
url = 'https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/10_minutes/air_temperature/recent/10minutenwerte_TU_01048_akt.zip'
new_data = pd.read_csv(url, sep=';')
new_data

,STATIONS_ID,MESS_DATUM,QN,PP_10,TT_10,TM5_10,RF_10,TD_10,eor
0,1048,201808010000,3,990.1,26.3,23.9,47.2,14.2,eor
1,1048,201808010010,3,990.1,26.2,23.9,48.0,14.3,eor
2,1048,201808010020,3,990.1,26.2,23.8,48.2,14.4,eor
3,1048,201808010030,3,990.1,25.8,23.5,50.1,14.6,eor
4,1048,201808010040,3,990.1,25.5,23.1,51.0,14.6,eor
...,...,...,...,...,...,...,...,...,...
79195,1048,202002012310,2,978.4,7.1,6.5,79.8,3.9,eor
79196,1048,202002012320,2,978.4,7.5,6.4,85.1,5.2,eor
79197,1048,202002012330,2,978.5,7.6,6.4,84.2,5.1,eor
79198,1048,202002012340,2,978.8,7.3,6.5,87.4,5.4,eor


## Step 3
- Adapt the downloaded data to the db data representation (datatypes, columns)

In [6]:
new_data.dtypes

STATIONS_ID      int64
MESS_DATUM       int64
  QN             int64
PP_10          float64
TT_10          float64
TM5_10         float64
RF_10          float64
TD_10          float64
eor             object
dtype: object

In [7]:
new_data.drop(['  QN', 'eor'], axis=1, inplace=True)
new_data.rename(columns={
    'MESS_DATUM': 'timestamp',
    'STATIONS_ID': 'id',
    'TT_10': 'temperature',
    'TM5_10': 'temperature_ground',
    'TD_10': 'temperature_dew',
    'RF_10': 'humidity',
    'PP_10': 'airpressure'}, inplace=True)
new_data = new_data.astype({'timestamp': str})
new_data = new_data[new_data['timestamp'] > last_original_row_timestamp]
new_data

,id,timestamp,airpressure,temperature,temperature_ground,humidity,temperature_dew
59697,1048,201909191330,1000.0,13.5,15.8,45.9,2.1
59698,1048,201909191340,1000.0,14.1,16.6,47.0,3.0
59699,1048,201909191350,1000.0,14.2,17.1,47.3,3.1
59700,1048,201909191400,1000.0,13.8,16.2,46.3,2.5
59701,1048,201909191410,1000.0,13.7,15.6,45.2,2.0
...,...,...,...,...,...,...,...
79195,1048,202002012310,978.4,7.1,6.5,79.8,3.9
79196,1048,202002012320,978.4,7.5,6.4,85.1,5.2
79197,1048,202002012330,978.5,7.6,6.4,84.2,5.1
79198,1048,202002012340,978.8,7.3,6.5,87.4,5.4


## Step 4
- Append the new data to the db and export the complete table to json

In [8]:
new_data.to_sql('dwd', connection, if_exists='append', index=False)
full_data = pd.concat([data_frame, new_data]).reset_index()
full_data.to_json('dwd.json')

- Generate the average temperature per hour in a new table

In [9]:
full_data['timestamp'] = pd.to_datetime(full_data['timestamp'])
average_per_hour = full_data.groupby(pd.DatetimeIndex(full_data.timestamp).hour).mean()
average_per_hour

,index,id,temperature,temperature_ground,temperature_dew,humidity,airpressure
timestamp,,,,,,,
0,68900.147059,1048.0,6.139338,5.008578,3.077328,81.471691,986.829902
1,68906.147059,1048.0,5.981005,4.843137,3.016667,82.053554,986.786887
2,68912.147059,1048.0,5.931373,4.797549,2.947181,81.983088,986.667647
3,68918.147059,1048.0,5.831618,4.737010,2.879167,82.164951,986.573039
4,68924.147059,1048.0,5.795588,4.739583,2.851961,82.198162,986.509559
5,68930.147059,1048.0,5.758824,4.706985,2.857108,82.382230,986.542157
6,68936.147059,1048.0,5.939583,5.107721,2.977083,82.022672,986.732966
7,68942.147059,1048.0,6.413358,5.993995,3.172672,80.578922,986.967647
8,68948.147059,1048.0,7.156495,7.261887,3.400613,77.978554,987.121078


In [10]:
average_per_hour.to_sql('average_per_hour', connection, if_exists='replace')

In [11]:
connection.close()